In [7]:
import numpy as np
import pandas as pd
from google.colab import drive
import sys

#For woring in Google Colab Directory
drive.mount('/content/gdrive/')
sys.path.append('/content/gdrive/My Drive/COMP 642/proj/')


import prepare_edr

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive/
Mounted at /content/gdrive/
[[7.60396040e-02 5.00000000e-02 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [7.60396040e-02 5.00000000e-02 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [7.60396040e-02 5.00000000e-02 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 ...
 [9.99026204e-01 8.00000000e-02 0.00000000e+00 ... 0.00000000e+00
  8.54400000e+02 0.00000000e+00]
 [9.99026204e-01 8.00000000e-02 0

In [0]:
#K-means clustering
test_data = prepare_edr.PrepareEDR("/content/gdrive/My Drive/COMP 642/proj/well_1.csv")

#Architecture contains the scaler so we can quickly back-out the original data if we wanted to.
X_train, X_headers, scaler = test_data.getClusteringTrainingData()


In [25]:
#From prior elbow modeling, number of clusters was decided as 8. Tune hyperparameters such as n_init and max_iterations (too high and we may be over-fitting)
from sklearn.model_selection import GridSearchCV
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN

n_clusters = 12

#variables to capture our optimal hyperparameters
min_inertia = float("inf")
n_init_opt = 0
max_iter_opt = 0

for i in range(10, 40, 10):
  for j in range(300, 600, 100):
    km = KMeans(n_clusters=n_clusters, init='k-means++', n_init=i, max_iter=j, random_state=0)
    km.fit(X_train)
    if km.inertia_ < min_inertia:
      min_inertia = km.inertia_
      n_init_opt = i
      max_iter_opt = j

##re-train with the optimal parameters; and then view and reason about the centroids.
print("k selection: " + str(n_clusters))
print("Optimal n_init value: " + str(n_init_opt))
print("Optimal max_iter value: " + str(max_iter_opt))
km_final = KMeans(n_clusters=n_clusters, init='k-means++', n_init=n_init_opt, max_iter=max_iter_opt, random_state=0)
km_final.fit(X_train)
centroids = km_final.cluster_centers_
centroid_params = scaler.inverse_transform(centroids)
centroid_params_df = pd.DataFrame(data = centroid_params, columns=X_headers)
centroid_params_df
#

k selection: 12
Optimal n_init value: 20
Optimal max_iter value: 300


,Bit Depth / Hole Depth,Rotary RPM,Weight on Bit,Total Pump Output,Block Movement: + / - / 0,Differential Pressure,Hook Load,On Bottom ROP,Standpipe Pressure,Convertible Torque
0,0.962344,3.330649,6.736251e+00,106.170051,-0.086509,5.506643e+00,231.474459,7.585994e-02,833.175263,2.243391
1,0.869198,3.174462,6.909480e-03,8.431173,0.238350,2.787359e-02,49.866899,1.072920e-12,92.269775,0.711041
2,0.999814,118.237526,3.292426e+01,341.514236,-0.983793,1.053529e+03,207.721016,2.933423e+02,5965.515565,17.475228
3,0.999346,56.052342,2.696333e+01,349.337967,-0.015588,1.818211e+02,209.037818,2.383066e+01,5161.780891,8.556837
4,0.111098,0.252955,7.460699e-13,0.103435,0.010804,5.499724e+00,45.371306,-2.366107e-12,17.868027,0.058247
5,0.999933,47.447762,1.884458e+01,307.307408,-0.993746,7.300548e+02,156.741939,1.366135e+02,3243.306744,8.397327
6,0.998965,27.867725,2.399334e+01,248.373205,0.025322,5.267619e+02,185.962541,2.878022e+01,2591.978716,7.634599
7,0.949093,115.923462,8.622651e+00,344.304067,-0.772201,1.027832e+02,228.076190,8.195946e+00,5102.611905,12.357017
8,0.933559,84.368642,3.196098e+00,314.959595,0.913295,8.927818e+01,261.958671,7.147110e-01,4465.994379,13.974162
9,0.999577,32.611829,1.332956e+01,290.161442,-0.016910,2.638235e+02,199.132775,1.718918e+01,3041.757067,6.403209
